# PyTorch EO Semantic Segmentation Example
## Step 4: Trained Model Conversion
*Rob Knapen, Wageningen Environmental Research*
<br>

This notebook illustrates two ways to convert a trained PyTorch model into a model that has no Python dependencies and can be used for inference in a different environment.

The most common approaches are converting (tracing) the model into TorchScript, and use of the Open Neural Network Exchange (ONNX) format.

In [12]:
import os
import h5py
import torch
import torch.nn as nn

In [20]:
# setup a path to the trained PyTorch model file
content_root_folder = os.path.join('..')

# need access to training data to get example inputs
hdf5_file = os.path.join(content_root_folder, 'data', 'archives', 'crops_training_data_s2_10m_2018seasons_224x224x28_76cat.hdf5')
codes_folder = os.path.join(content_root_folder, 'data', 'raw')
codes_file = os.path.join(codes_folder, 'cell_value_to_crop_info_76_classes.csv')

base_model_filename = 'rvo_crops_segnet_224x224x28_77classes_100epochs'

# the trained model to convert
pytorch_trained_model_file = os.path.join(content_root_folder, 'models', base_model_filename + '_model_full.pt')

# the output TorchScript model
torchscript_trained_model_file = os.path.join(content_root_folder, 'models', base_model_filename + '_model_full_traced.pt')

# the output ONNX model
onnx_trained_model_file = os.path.join(content_root_folder, 'models', base_model_filename + '_model_full.onnx')

In [14]:
# need the network class definition

class SegNet(nn.Module):
    """
    EncoderDecoder network for semantic segmentation
    """

    def __init__(self, n_channels, encoder_conv_width, decoder_conv_width, n_class, cuda = 0):
        """
        initialization function
        n_channels, int, number of input channel
        encoder_conv_width, int list, size of the feature maps depth for the encoder after each conv
        decoder_conv_width, int list, size of the feature maps depth for the decoder after each conv
        n_class = int,  the number of classes
        """
        super(SegNet, self).__init__() # necessary for all classes extending the module class

        assert((encoder_conv_width[3] == encoder_conv_width[5]) \
               and (encoder_conv_width[1] == decoder_conv_width[1]))

        self.maxpool=nn.MaxPool2d(2,2,return_indices=True) # maxpooling layer
        self.unpool=nn.MaxUnpool2d(2,2) # unpooling layer

        # encoder
        self.c1 = nn.Sequential(
            nn.Conv2d(n_channels, encoder_conv_width[0], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[0]),
            nn.ReLU(True))
        self.c2 = nn.Sequential(
            nn.Conv2d(encoder_conv_width[0], encoder_conv_width[1], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[1]),
            nn.ReLU(True))
        self.c3 = nn.Sequential(
            nn.Conv2d(encoder_conv_width[1], encoder_conv_width[2], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[2]),
            nn.ReLU(True))
        self.c4 = nn.Sequential(
            nn.Conv2d(encoder_conv_width[2], encoder_conv_width[3], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[3]),
            nn.ReLU(True))
        self.c5 = nn.Sequential(
            nn.Conv2d(encoder_conv_width[3], encoder_conv_width[4], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[4]),
            nn.ReLU(True))
        self.c6 = nn.Sequential(
            nn.Conv2d(encoder_conv_width[4], encoder_conv_width[5], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(encoder_conv_width[5]),
            nn.ReLU(True))

        # decoder
        self.c7=nn.Sequential(
            nn.Conv2d(encoder_conv_width[5] + encoder_conv_width[3], decoder_conv_width[0], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(decoder_conv_width[0]),
            nn.ReLU(True))
        self.c8=nn.Sequential(
            nn.Conv2d(decoder_conv_width[0], decoder_conv_width[1], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(decoder_conv_width[1]),
            nn.ReLU(True))
        self.c9=nn.Sequential(
            nn.Conv2d(encoder_conv_width[1] + decoder_conv_width[1], decoder_conv_width[2], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(decoder_conv_width[2]),
            nn.ReLU(True))
        self.c10=nn.Sequential(
            nn.Conv2d(decoder_conv_width[2], decoder_conv_width[3], 3, padding=1, padding_mode='reflect'),
            nn.BatchNorm2d(decoder_conv_width[3]),
            nn.ReLU(True)) # for regularisation could add dropout here, nn.Dropout()

        # final classifying layer
        self.classifier=nn.Conv2d(decoder_conv_width[3], n_class, 3, padding=1, padding_mode='reflect')

        # weight initialization
        self.c1[0].apply(self.init_weights)
        self.c2[0].apply(self.init_weights)
        self.c3[0].apply(self.init_weights)
        self.c4[0].apply(self.init_weights)
        self.c5[0].apply(self.init_weights)
        self.c6[0].apply(self.init_weights)
        self.c7[0].apply(self.init_weights)
        self.c8[0].apply(self.init_weights)
        self.c9[0].apply(self.init_weights)
        self.c10[0].apply(self.init_weights)
        self.classifier.apply(self.init_weights)

        if cuda: # put the model on the GPU memory
            self.cuda()

    def init_weights(self,layer): # gaussian init for the conv layers
        nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')

    def forward(self,input):
        """
        the function called to run inference
        """
        # encoder
        #level 1
        x1 = self.c2(self.c1(input))
        x2, indices_a_b = self.maxpool(x1)
        #level 2
        x3= self.c4(self.c3(x2))
        x4, indices_b_c =self.maxpool(x3)
        #level 3
        x5 = self.c6(self.c5(x4))
        # decoder
        #level 2
        y4 = self.unpool(x5, indices_b_c, x2.size())
        y3 = self.c8(self.c7(torch.cat((y4,x3), 1)))
        # level 1
        y2 = self.unpool(y3, indices_a_b, x1.size())
        y1 = self.c10(self.c9(torch.cat((y2,x1), 1)))
        # output
        out = self.classifier(y1)
        return out

In [15]:
# need example data for the conversion
data_file = h5py.File(hdf5_file, "r")

print("(Attribute, Value:)");
for item in data_file.attrs.items():
    print(item);

train_obs = data_file['train_observation'][:]
train_gt = data_file['train_groundtruth'][:]
test_obs = data_file['test_observation'][:]
test_gt = data_file['test_groundtruth'][:]
n_train = train_obs.shape[0]
n_test = test_obs.shape[0]

print("%d tiles for training, %d tiles for testing" % (n_train, n_test))

(Attribute, Value:)
('description', 'This data set is intended for training deep learning models for crop classification.')
('ground_truth', 'Encoded crop types for fields according to the RVO registration in 2018.')
('ground_truth_shape', '1,224,224; 76 classes, int16')
('observations', '4 Bands (blue, green, red, nir) of 7 seasonal Sentinel 2 images, from 2018.')
('observations_shape', '28,224,224; scaled [0-1], float32')
('spatial_extent', 'Sample area around the Noordoost polder in The Netherlands.')
('spatial_resolution', '10x10m grid cells')
('title', 'Crop classification training data set')
151 tiles for training, 43 tiles for testing


In [16]:
# load the entire model (on CPU)
model = torch.load(pytorch_trained_model_file, map_location=torch.device('cpu'))
model.eval()
model

SegNet(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unpool): MaxUnpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  (c1): Sequential(
    (0): Conv2d(28, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (c2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (c3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (c4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=ref

## TorchScript Conversion

In [38]:
# Convert the trained model to TorchScript

# create a tensor with example input data
tile_index = 1
tile = train_obs[tile_index,:,:,:].transpose(2,0,1)
tile_t = torch.from_numpy(tile).unsqueeze(dim=0)

# trace the model to a TorchScript version
traced_model = torch.jit.trace(model, tile_t)
traced_model

SegNet(
  original_name=SegNet
  (maxpool): MaxPool2d(original_name=MaxPool2d)
  (unpool): MaxUnpool2d(original_name=MaxUnpool2d)
  (c1): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): ReLU(original_name=ReLU)
  )
  (c2): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): ReLU(original_name=ReLU)
  )
  (c3): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): ReLU(original_name=ReLU)
  )
  (c4): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): ReLU(original_name=ReLU)
  )
  (c5): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): ReLU(original_name=ReLU)
  )
  (c6

In [39]:
traced_model.state_dict()

OrderedDict([('c1.0.weight',
              tensor([[[[ 0.0543, -0.0334, -0.0157],
                        [ 0.0239,  0.0966,  0.0349],
                        [ 0.0629, -0.1413, -0.0657]],
              
                       [[ 0.1023, -0.0224, -0.0558],
                        [-0.1220, -0.0497, -0.1010],
                        [-0.1012, -0.0448, -0.0014]],
              
                       [[ 0.0176,  0.0130,  0.0839],
                        [ 0.0128, -0.0361,  0.1166],
                        [-0.0032,  0.0057, -0.0356]],
              
                       ...,
              
                       [[ 0.0121,  0.0290, -0.0348],
                        [-0.0173,  0.0434, -0.0268],
                        [-0.0860, -0.0691, -0.0090]],
              
                       [[-0.0570, -0.0615, -0.0435],
                        [-0.0086, -0.0417,  0.0436],
                        [ 0.0361, -0.0283,  0.0531]],
              
                       [[ 0.0164, -0.0293, -0.0069],


In [21]:
# save the traced TorchScript model (can be loaded into libtorch (C++))
traced_model.save(torchscript_trained_model_file)

## ONNX Conversion
See: https://onnxruntime.ai/docs/tutorials/accelerate-pytorch/pytorch.html#convert-model-to-onnx

In [37]:
# export the trained model to ONNX
from torch.onnx import TrainingMode

# create a tensor with example input data
tile_index = 1
tile = train_obs[tile_index,:,:,:].transpose(2,0,1)
tile_t = torch.from_numpy(tile).unsqueeze(dim=0)

# export it (can take a while...) -- currently causes some issues ...
torch.onnx.export(
    model, tile_t, onnx_trained_model_file,
    opset_version=16,
    verbose=True,
    training=TrainingMode.EVAL)

UnsupportedOperatorError: Exporting the operator ::max_unpool2d to ONNX opset version 16 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub.